In [1]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

from qiskit.circuit.library import n_local
from qiskit.circuit import QuantumCircuit

In [2]:
num_qubits=5



# device='lightning.qubit'
device='qulacs.simulator'

device = qml.device(device, wires=num_qubits)


In [3]:
class EfficientSU2:
    def __init__(self,
                 wires: list = None,
                 su2_gates: list = None,
                 entanglement: str = 'linear',
                 reps: int = 1,
                 skip_final_rot: bool = False
                 ):
        if su2_gates is None:
            su2_gates = ['ry', 'rz']
        self._su2_gates = su2_gates
        self._entanglement = entanglement
        self._reps = reps
        self._skip_final_rot = skip_final_rot
        self._wires = None
        self._qc = None
        if wires is not None:
            self.set_wires(wires)
            
    
    def __call__(self, parameters, wires: list = None):
        if wires is not None:
            
            if len(self._wires) != len(self._wires):
                raise ValueError("Cannot override wires instance of different length")
            self._wires = wires
            
        qc = self._qc
        if qc.num_parameters_settable != len(parameters):
            raise ValueError("Incorrect number of parameters. Expected ", qc.num_parameters_settable, 'but received ',
                             len(parameters))
        qc = qc.decompose()
        parameters = parameters.tolist()
        qc = qc.assign_parameters(parameters)
        qml_circuit = qml.from_qiskit(qc)
        qml_circuit(wires=self._wires)
        # qml.draw_mpl(qml_circuit)()
        
    @property
    def num_params(self):
        return self._qc.num_parameters_settable

    @property
    def layers(self):
        return self._reps

    @layers.setter
    def layers(self, val):
        self._reps = val
    
    def set_wires(self, wires):
        self._wires = wires
        self._qc = n_local.EfficientSU2(num_qubits=len(self._wires), su2_gates=self._su2_gates,
                                        entanglement=self._entanglement, reps=self._reps,
                                        skip_final_rotation_layer=self._skip_final_rot)


In [4]:

variational=EfficientSU2(skip_final_rot=True)
LAYERS=1
re_upload_depth=1
wires=range(num_qubits)
variational.layers = LAYERS
variational.set_wires(wires)

num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, num_qubits)
# variational(initial_parameters)
def circuit(parameters, wires):
    variational(parameters, wires)
    return  qml.expval(qml.PauliZ(0))
for i in range(2):
    try:    
        print('FUcked')
        shit = qml.QNode(circuit,device)(parameters=initial_parameters, wires=wires)
    except:
        print('Not Fucked')
        shit = qml.QNode(circuit,device)(parameters=initial_parameters, wires=wires)
    print(shit)
# qml.draw_mpl(variational)(parameters=initial_parameters, wires=wires)
# plt.title("YIKES")
# plt.show()

FUcked
Not Fucked
-0.9717148816314164
FUcked
-0.9717148816314164


In [5]:
shit

array(-0.97171488)